라이브러리 및 제출양식함수

In [ ]:
!pip install xgboost
!pip install catboost
!pip install optuna
!pip install autogluon

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
# from catboost import CatBoostRegressor
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import KFold, cross_val_score
# import optuna
from sklearn.preprocessing import PolynomialFeatures

import numpy as np
# import autogluon as ag
# from autogluon.tabular import TabularDataset, TabularPredictor
# from autogluon.core.metrics import make_scorer

In [ ]:
# 제출양식 함수(베이스라인)

def submitResult(pred):
    try:
        label = pd.read_csv('/content/drive/MyDrive/kt1/Q1_label_sample.csv')
        # 1. 컬럼명과 순서가 동일한지 체크
        if pred.columns.equals(label.columns):
            print("Check: 컬럼명과 순서가 동일합니다.")
        else:
            print(f"Warning: 컬럼명과 순서가 동일하지 않습니다.\n- 예측 데이터프레임 컬럼명: {pred.columns}\n- 레이블 데이터프레임 컬럼명: {label.columns}")
            return

        # 2. datetime 컬럼이 존재하며 해당 컬럼의 샘플수와 값이 일치하는지 체크
        if (label['datetime'] == pred['datetime']).all():
            print("Check: datetime 순서와 샘플 수가 일치합니다.")
        else:
            print("Warning: 테스트 세트와 모델 예측의 datetime이 일치하지 않습니다.")
            return

        pred.to_csv('Q1_submitResult.csv', index=False)
        print("Done : Q1_submitResult.csv 파일로 저장되었습니다.")
    except Exception as e:
        # 예외가 발생한 경우 오류 메시지 출력
        print("Error:", e)

데이터로드

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/kt1/Q1_train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/kt1/Q1_test.csv')
test_df = test_df.drop('Unnamed: 0',axis=1)
label_sample = pd.read_csv('/content/drive/MyDrive/kt1/Q1_label_sample.csv')

In [ ]:
pd.DataFrame(train_df[['uenomax']].value_counts(normalize=True))

,0
uenomax,
2,0.207683
3,0.194660
4,0.150206
1,0.136673
5,0.104216
...,...
57,0.000007
43,0.000007
61,0.000007


결측치처리

In [ ]:
# 결측치 중위수대체
train_df = pd.read_csv('/content/drive/MyDrive/kt1/Q1_train.csv')
train_df = train_df.iloc[:,2:]
train_df = train_df.fillna(train_df.median())
X = train_df.drop('uenomax',axis=1)
y = train_df['uenomax']
# LightGBM 모델 생성
model = lgb.LGBMRegressor(random_state=42)

# 교차 검증 설정
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = make_scorer(mean_absolute_error)
cv_scores = cross_val_score(model, X, y, cv=kf,scoring=scoring)

# 전체 교차 검증 결과 출력
print(f"전체 MAE: {cv_scores}")
print(f"평균 MAE: {np.mean(cv_scores)}")
print(f"MAE 표준편차: {np.std(cv_scores)}")

전체 MAE: [0.63462114 0.63737791 0.62989381 0.62810787 0.63142982]
평균 MAE: 0.6322861087599998
MAE 표준편차: 0.0033254449907759182


In [ ]:
# 결측치를 dropna
train_df = pd.read_csv('/content/drive/MyDrive/kt1/Q1_train.csv')
train_df = train_df.iloc[:,2:]
train_df = train_df.dropna()
train_df = train_df.reset_index(drop=True) # 인덱스정렬
X = train_df.drop('uenomax',axis=1)
y = train_df['uenomax']
# LightGBM 모델 생성
model = lgb.LGBMRegressor(random_state=42)

# 교차 검증 설정
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = make_scorer(mean_absolute_error)
cv_scores = cross_val_score(model, X, y, cv=kf,scoring=scoring)

# 전체 교차 검증 결과 출력
print(f"전체 MAE: {cv_scores}")
print(f"평균 MAE: {np.mean(cv_scores)}")
print(f"MAE 표준편차: {np.std(cv_scores)}")

전체 MAE: [0.62994744 0.63305965 0.62704125 0.63406055 0.6348694 ]
평균 MAE: 0.6317956591167769
MAE 표준편차: 0.0029050157606398215


In [ ]:
# 따라서 결측치는 drop처리

피처엔지니어링 - 연도

In [ ]:
# 결측치를 dropna + 연도피처엔지니어링(기존 datetime 컬럼 유지)
train_df = pd.read_csv('/content/drive/MyDrive/kt1/Q1_train.csv')
train_df = train_df.drop(columns='ru_id')
train_df = train_df.dropna()
train_df = train_df.reset_index(drop=True) # 인덱스정렬

# 연도, 월, 일, 시간, 분 컬럼 추가
train_df['datetime'] = pd.to_datetime(train_df['datetime'])

train_df['year'] = train_df['datetime'].dt.year
train_df['month'] = train_df['datetime'].dt.month
train_df['day'] = train_df['datetime'].dt.day
train_df['hour'] = train_df['datetime'].dt.hour
train_df['minute'] = train_df['datetime'].dt.minute

train_df['datetime'] = train_df['datetime'].astype(int)

X = train_df.drop('uenomax',axis=1)
y = train_df['uenomax']
# LightGBM 모델 생성
model = lgb.LGBMRegressor(random_state=42)

# 교차 검증 설정
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = make_scorer(mean_absolute_error)
cv_scores = cross_val_score(model, X, y, cv=kf,scoring=scoring)

# 전체 교차 검증 결과 출력
print(f"전체 MAE: {cv_scores}")
print(f"평균 MAE: {np.mean(cv_scores)}")
print(f"MAE 표준편차: {np.std(cv_scores)}")

전체 MAE: [0.62658055 0.6305869  0.62503971 0.63155685 0.63135563]
평균 MAE: 0.6290239272073956
MAE 표준편차: 0.002688475830051447


In [ ]:
# 결측치를 dropna + 연도피처엔지니어링(기존 datetime 컬럼 삭제)
train_df = pd.read_csv('/content/drive/MyDrive/kt1/Q1_train.csv')
train_df = train_df.drop(columns='ru_id')
train_df = train_df.dropna()
train_df = train_df.reset_index(drop=True) # 인덱스정렬

# 연도, 월, 일, 시간, 분 컬럼 추가
train_df['datetime'] = pd.to_datetime(train_df['datetime'])

train_df['year'] = train_df['datetime'].dt.year
train_df['month'] = train_df['datetime'].dt.month
train_df['day'] = train_df['datetime'].dt.day
train_df['hour'] = train_df['datetime'].dt.hour
train_df['minute'] = train_df['datetime'].dt.minute

train_df = train_df.drop(columns='datetime')

X = train_df.drop('uenomax',axis=1)
y = train_df['uenomax']
# LightGBM 모델 생성
model = lgb.LGBMRegressor(random_state=42)

# 교차 검증 설정
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = make_scorer(mean_absolute_error)
cv_scores = cross_val_score(model, X, y, cv=kf,scoring=scoring)

# 전체 교차 검증 결과 출력
print(f"전체 MAE: {cv_scores}")
print(f"평균 MAE: {np.mean(cv_scores)}")
print(f"MAE 표준편차: {np.std(cv_scores)}")

전체 MAE: [0.62730661 0.63125357 0.62515487 0.63180413 0.63207364]
평균 MAE: 0.6295185648129742
MAE 표준편차: 0.002781980788262308


In [ ]:
# 따라서 연도피처엔지니어링(기존 datetime 컬럼 유지) 채택

피처엔지니어링 - train데이터셋에 test데이터셋의 데이터(ru_id) 추가

In [ ]:
#데이터로드 및 전처리

train_df = pd.read_csv('/content/drive/MyDrive/kt1/Q1_train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/kt1/Q1_test.csv')
test_df = test_df.drop('Unnamed: 0',axis=1)
label_sample = pd.read_csv('/content/drive/MyDrive/kt1/Q1_label_sample.csv')

# dropna로 결측치처리
train_df = train_df.dropna()
train_df = train_df.reset_index(drop=True) # 인덱스정렬


# 연도, 월, 일, 시간, 분 컬럼 추가
train_df['datetime'] = pd.to_datetime(train_df['datetime'])

train_df['year'] = train_df['datetime'].dt.year
train_df['month'] = train_df['datetime'].dt.month
train_df['day'] = train_df['datetime'].dt.day
train_df['hour'] = train_df['datetime'].dt.hour
train_df['minute'] = train_df['datetime'].dt.minute

train_df['datetime'] = train_df['datetime'].astype(int)



test_df['datetime'] = pd.to_datetime(test_df['datetime'])

test_df['year'] = test_df['datetime'].dt.year
test_df['month'] = test_df['datetime'].dt.month
test_df['day'] = test_df['datetime'].dt.day
test_df['hour'] = test_df['datetime'].dt.hour
test_df['minute'] = test_df['datetime'].dt.minute

test_df['datetime'] = test_df['datetime'].astype(int)



X = train_df.drop('uenomax',axis=1)
y = train_df['uenomax']

temp = test_df.iloc[34360:34362,:]
X = pd.concat([X, temp], ignore_index=True)
y[137436] = np.median(y)
y[137437] = np.median(y)

# 정규화
scaler = StandardScaler()
X.iloc[:,2:-5] = scaler.fit_transform(X.iloc[:,2:-5])

test_df.iloc[:,2:-5] = scaler.transform(test_df.iloc[:,2:-5])

# ru_id 라벨 인코딩
label_encoder = LabelEncoder()
X['ru_id'] = label_encoder.fit_transform(X['ru_id'])
test_df['ru_id'] = label_encoder.transform(test_df['ru_id'])

# LightGBM 모델 생성
model = lgb.LGBMRegressor(random_state=42)

# 교차 검증 설정
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = make_scorer(mean_absolute_error)
cv_scores = cross_val_score(model, X, y, cv=kf,scoring=scoring)

# 전체 교차 검증 결과 출력
print(f"전체 MAE: {cv_scores}")
print(f"평균 MAE: {np.mean(cv_scores)}")
print(f"MAE 표준편차: {np.std(cv_scores)}")

<ipython-input-81-00b70c77a254>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[137436] = np.median(y)


전체 MAE: [0.6233114  0.62335067 0.62231809 0.62733159 0.62731564]
평균 MAE: 0.624725476515296
MAE 표준편차: 0.002153413667863317


In [ ]:
# 따라서 위와 같은 방식 최종 채택

모델탐색

In [ ]:
# xgboost
model = xgb.XGBRegressor(random_state=42)
# 교차 검증 설정
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = make_scorer(mean_absolute_error)
cv_scores = cross_val_score(model, X, y, cv=kf,scoring=scoring)

# 전체 교차 검증 결과 출력
print(f"전체 MAE: {cv_scores}")
print(f"평균 MAE: {np.mean(cv_scores)}")
print(f"MAE 표준편차: {np.std(cv_scores)}")

전체 MAE: [0.62110142 0.62106884 0.61804372 0.62659766 0.62617796]
평균 MAE: 0.6225979195862863
MAE 표준편차: 0.0032903765694001536


In [ ]:
# CatBoostRegressor
model = CatBoostRegressor(verbose=0,random_state=42)
# 교차 검증 설정
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = make_scorer(mean_absolute_error)
cv_scores = cross_val_score(model, X, y, cv=kf,scoring=scoring)

# 전체 교차 검증 결과 출력
print(f"전체 MAE: {cv_scores}")
print(f"평균 MAE: {np.mean(cv_scores)}")
print(f"MAE 표준편차: {np.std(cv_scores)}")

전체 MAE: [0.60867496 0.60815976 0.60538186 0.61377033 0.61382293]
평균 MAE: 0.6099619697398875
MAE 표준편차: 0.0033254288069213037


In [ ]:
# 따라서 CatBoostRegressor 채택

하이퍼 파라메터 튜닝 - OPTUNA

In [ ]:
# Objective 함수 정의
def objective(trial):
    # 하이퍼파라미터 탐색 범위 설정
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.1, 10.0),
        'random_strength': trial.suggest_float('random_strength', 0.1, 1.0),
    }

    # CatBoostRegressor 모델 초기화
    model = CatBoostRegressor(verbose=0,random_state=42, **params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 평가 지표 (MAE) 계산
    mae = mean_absolute_error(y_test, model.predict(x_test))

    return mae

# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Optuna 최적화 설정
study = optuna.create_study(direction='minimize')

# 최적화 진행 상황 출력
def print_study_statistics(study, trial):
    print(f'Trial {trial.number}/{len(study.trials)} - Loss: {trial.value:.4f} - Best Loss: {study.best_value:.4f} - Params: {trial.params}')

study.optimize(objective, n_trials=100, callbacks=[print_study_statistics])

# 최적 하이퍼파라미터 출력
print('Best Parameters:', study.best_params)
print('Best MAE:', study.best_value)

In [ ]:
# CatBoostRegressor
model = CatBoostRegressor(verbose=0, random_state=42, learning_rate=0.08807070244833717, depth=10, l2_leaf_reg=6.861317113568743, random_strength=0.2220645202847003)
# 교차 검증 설정
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = make_scorer(mean_absolute_error)
cv_scores = cross_val_score(model, X, y, cv=kf,scoring=scoring)

# 전체 교차 검증 결과 출력
print(f"전체 MAE: {cv_scores}")
print(f"평균 MAE: {np.mean(cv_scores)}")
print(f"MAE 표준편차: {np.std(cv_scores)}")

전체 MAE: [0.60529284 0.60510462 0.6010728  0.61030595 0.60975552]
평균 MAE: 0.6063063486051441
MAE 표준편차: 0.003398674642087573


In [ ]:
model = CatBoostRegressor(verbose=0, random_state=42, learning_rate=0.08807070244833717, depth=10, l2_leaf_reg=6.861317113568743, random_strength=0.2220645202847003)
model.fit(X,y)

y_pred = model.predict(test_df)

test_df = pd.read_csv('/content/drive/MyDrive/kt1/Q1_test.csv')
test_df['uenomax'] = y_pred
df = test_df[['datetime','ru_id','uenomax']]
y2 = df.pivot_table(index='datetime', columns='ru_id', values='uenomax', fill_value=0)
y2.columns.name = None
y2 = y2.reset_index()

submitResult(y2)

In [ ]:
# poly(degree=2) 적용
pf = PolynomialFeatures(degree=2)
X_poly = pf.fit_transform(X)

# CatBoostRegressor
model = CatBoostRegressor(verbose=0, random_state=42, learning_rate=0.08807070244833717, depth=10, l2_leaf_reg=6.861317113568743, random_strength=0.2220645202847003)
# 교차 검증 설정
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scoring = make_scorer(mean_absolute_error)
cv_scores = cross_val_score(model, X_poly, y, cv=kf,scoring=scoring)

# 전체 교차 검증 결과 출력
print(f"전체 MAE: {cv_scores}")
print(f"평균 MAE: {np.mean(cv_scores)}")
print(f"MAE 표준편차: {np.std(cv_scores)}")

전체 MAE: [0.61005831 0.61176741 0.60819656 0.61544919 0.6159599 ]
평균 MAE: 0.6122862739454841
MAE 표준편차: 0.0030152388678059763


In [ ]:
# poly(degree=2) 적용
pf = PolynomialFeatures(degree=2)
X_poly = pf.fit_transform(X)

model = CatBoostRegressor(verbose=0, random_state=42, learning_rate=0.08807070244833717, depth=10, l2_leaf_reg=6.861317113568743, random_strength=0.2220645202847003)
model.fit(X_poly,y)

test_df_poly = pf.transform(test_df)
y_pred = model.predict(test_df_poly)

test_df = pd.read_csv('/content/drive/MyDrive/kt1/Q1_test.csv')
test_df['uenomax'] = y_pred
df = test_df[['datetime','ru_id','uenomax']]
y = df.pivot_table(index='datetime', columns='ru_id', values='uenomax', fill_value=0)
y.columns.name = None
y = y.reset_index()

submitResult(y)

Autogluon

Autogluon - 회귀

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/kt1/Q1_train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/kt1/Q1_test.csv')
test_df = test_df.drop('Unnamed: 0',axis=1)
label_sample = pd.read_csv('/content/drive/MyDrive/kt1/Q1_label_sample.csv')

# 데이터셋 로드
train_data = train_df

# 회귀하려는 타겟 변수 지정
label = 'uenomax'

# 오토 글루온으로 회귀 모델 학습
predictor = TabularPredictor(label=label, problem_type='regression')
predictor.fit(train_data, presets='best', time_limit=3600*5)

# 테스트 데이터 로드 (예시로 test.csv 파일 사용)
# 이 데이터에는 y값이 없어야 합니다.
test_data = test_df

# 학습된 모델로 예측 수행
y_pred = predictor.predict(test_data)

test_data = pd.read_csv('/content/drive/MyDrive/kt1/Q1_test.csv')
test_data['uenomax'] = y_pred
df = test_data[['datetime', 'ru_id', 'uenomax']]
y2=df.pivot_table(index='datetime', columns='ru_id', values='uenomax', fill_value=0)

y2.columns.name = None
y2 = y2.reset_index()


# 확인 및 저장
submitResult(y2)

Autogluon - 다중클래스분류

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/kt1/Q1_train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/kt1/Q1_test.csv')
test_df = test_df.drop('Unnamed: 0',axis=1)
label_sample = pd.read_csv('/content/drive/MyDrive/kt1/Q1_label_sample.csv')

# 데이터셋 로드
train_data = train_df

# 회귀하려는 타겟 변수 지정
label = 'uenomax'

ag_accuracy_scorer = make_scorer(name='mae2',
                                 score_func=mean_absolute_error,
                                 greater_is_better=False)


# 오토 글루온으로 회귀 모델 학습
predictor = TabularPredictor(label=label, problem_type='multiclass',eval_metric=ag_accuracy_scorer)
predictor.fit(train_data,  presets='best_quality',time_limit=3600 *1.5,)

# 테스트 데이터 로드 (예시로 test.csv 파일 사용)
# 이 데이터에는 y값이 없어야 합니다.
test_data = test_df

# 학습된 모델로 예측 수행
y_pred = predictor.predict(test_data)

test_data = pd.read_csv('/content/drive/MyDrive/kt1/Q1_test.csv')
test_data['uenomax'] = y_pred
df = test_data[['datetime', 'ru_id', 'uenomax']]
y2=df.pivot_table(index='datetime', columns='ru_id', values='uenomax', fill_value=0)

y2.columns.name = None
y2 = y2.reset_index()


# 확인 및 저장
submitResult(y2)